<a href="https://colab.research.google.com/github/Ritikseptember2003/Improved-YOLOv8_Foggy_Image_Object_Detection/blob/dev/Improved_YOLOv8_Foggy_Image_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics torch torchvision torchaudio --quiet
!pip install opencv-python albumentations timm tqdm --quiet
!pip install onnx onnxruntime --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
from IPython.display import clear_output
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .
clear_output()
print("YOLOv8 installed successfully!")

YOLOv8 installed successfully!


In [ ]:
import torch
print("Torch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU Found")

Torch Version: 2.6.0+cu124
CUDA Available: True
GPU Name: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/RTTS.zip /content/

In [ ]:
import zipfile
import os

zip_path = "/content/RTTS.zip"
extract_path = "/content/datasets"

# Unzip
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset successfully extracted to /content/datasets/")

Dataset successfully extracted to /content/datasets/


In [ ]:
!ls /content/datasets

Annotations  ImageSets	JPEGImages


In [ ]:
!pip install lxml tqdm --quiet

In [ ]:
import os

yolo_labels_path = "/content/datasets/labels"
os.makedirs(yolo_labels_path, exist_ok=True)
os.makedirs(f"{yolo_labels_path}/train", exist_ok=True)
os.makedirs(f"{yolo_labels_path}/val", exist_ok=True)
os.makedirs(f"{yolo_labels_path}/test", exist_ok=True)

print("YOLO labels directory created!")

YOLO labels directory created!


In [ ]:
!ls /content/datasets

Annotations  ImageSets	JPEGImages  labels


In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
from tqdm import tqdm

# Define paths
voc_annotations_path = "/content/datasets/Annotations"
yolo_labels_path = "/content/datasets/labels/train"

# Define class names (same as the research paper)
class_names = ["car", "bus", "person", "bicycle", "motorbike"]

# Function to convert XML to YOLO format
def convert_voc_to_yolo(xml_file, output_txt_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    img_width = int(root.find("size/width").text)
    img_height = int(root.find("size/height").text)

    with open(output_txt_file, "w") as f:
        for obj in root.findall("object"):
            class_name = obj.find("name").text
            if class_name not in class_names:
                continue  # Skip unknown classes

            class_id = class_names.index(class_name)
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            # Convert to YOLO format (normalized)
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Convert all XML files
xml_files = glob.glob(f"{voc_annotations_path}/*.xml")
for xml_file in tqdm(xml_files, desc="Converting Annotations"):
    output_txt_file = f"{yolo_labels_path}/{os.path.basename(xml_file).replace('.xml', '.txt')}"
    convert_voc_to_yolo(xml_file, output_txt_file)

print(f"Converted {len(xml_files)} XML annotations to YOLO format!")

Converting Annotations: 100%|██████████| 4322/4322 [00:01<00:00, 3169.79it/s]

Converted 4322 XML annotations to YOLO format!


In [ ]:
!ls /content/datasets/labels/

test  train  val


In [ ]:
!ls /content/datasets/labels/train | head -10

AM_Bing_211.txt
AM_Bing_217.txt
AM_Bing_222.txt
AM_Bing_229.txt
AM_Bing_232.txt
AM_Bing_242.txt
AM_Bing_243.txt
AM_Bing_274.txt
AM_Bing_318.txt
AM_Bing_422.txt


In [ ]:
import os

# Define dataset directories
split_dirs = ["train", "val", "test"]
for split in split_dirs:
    os.makedirs(f"/content/datasets/images/{split}", exist_ok=True)
    os.makedirs(f"/content/datasets/labels/{split}", exist_ok=True)

print("Created Train, Validation, and Test folders!")

Created Train, Validation, and Test folders!


In [ ]:
!ls /content/datasets/

Annotations  images  ImageSets	JPEGImages  labels


In [ ]:
!ls /content/datasets/images/

test  train  val


In [ ]:
!ls /content/datasets/JPEGImages | head -10

AM_Bing_211.png
AM_Bing_217.png
AM_Bing_222.png
AM_Bing_229.png
AM_Bing_232.png
AM_Bing_242.png
AM_Bing_243.png
AM_Bing_274.png
AM_Bing_318.png
AM_Bing_422.png


In [ ]:
import glob
import shutil
import os
import random

# Update image directory path
image_dir = "/content/datasets/JPEGImages"  # Confirm this is correct
label_dir = "/content/datasets/labels/train"

# Check if directory exists
if not os.path.exists(image_dir):
    print(f"ERROR: Image directory {image_dir} not found!")
else:
    print(f"Image directory found: {image_dir}")

# Get all PNG images
image_files = glob.glob(f"{image_dir}/*.png")  # Updated for PNG files
if len(image_files) == 0:
    print("No images found in directory!")
else:
    print(f"✅ Found {len(image_files)} images")

# Shuffle dataset randomly
random.shuffle(image_files)

# Define split ratios
train_ratio, val_ratio = 0.8, 0.1
train_split = int(len(image_files) * train_ratio)
val_split = int(len(image_files) * (train_ratio + val_ratio))

# Split dataset
train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

# Function to move images & labels
def move_files(files, split):
    for file in files:
        file_name = os.path.basename(file)
        img_dest = f"/content/datasets/images/{split}/{file_name}"
        label_file = f"{label_dir}/{file_name.replace('.png', '.txt')}"  # 🔹 Updated for PNG

        # Move image
        if os.path.exists(file):
            shutil.move(file, img_dest)

        # Move corresponding label
        if os.path.exists(label_file):
            shutil.move(label_file, f"/content/datasets/labels/{split}/{file_name.replace('.png', '.txt')}")

# Move images & labels into correct folders
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print(f"Fixed Dataset Split: {len(train_files)} Train, {len(val_files)} Val, {len(test_files)} Test")

Image directory found: /content/datasets/JPEGImages
✅ Found 4322 images
Fixed Dataset Split: 3457 Train, 432 Val, 433 Test


In [ ]:
print("Train Images:", len(os.listdir("/content/datasets/images/train")))
print("Validation Images:", len(os.listdir("/content/datasets/images/val")))
print("Test Images:", len(os.listdir("/content/datasets/images/test")))

print("Train Labels:", len(os.listdir("/content/datasets/labels/train")))
print("Validation Labels:", len(os.listdir("/content/datasets/labels/val")))
print("Test Labels:", len(os.listdir("/content/datasets/labels/test")))

Train Images: 3457
Validation Images: 432
Test Images: 433
Train Labels: 3457
Validation Labels: 432
Test Labels: 433


In [ ]:
yaml_content = """
train: /content/datasets/images/train
val: /content/datasets/images/val
test: /content/datasets/images/test

nc: 5  # Number of classes
names: ["car", "bus", "person", "bicycle", "motorbike"]
"""

with open("/content/datasets/data.yaml", "w") as f:
    f.write(yaml_content)

print("YOLOv8 dataset configuration file `data.yaml` created!")

YOLOv8 dataset configuration file `data.yaml` created!


In [ ]:
!cat /content/datasets/data.yaml


train: /content/datasets/images/train
val: /content/datasets/images/val
test: /content/datasets/images/test

nc: 5  # Number of classes
names: ["car", "bus", "person", "bicycle", "motorbike"]


In [ ]:
!pip uninstall ultralytics -y
!pip install ultralytics --upgrade --quiet

Found existing installation: ultralytics 8.3.92
Uninstalling ultralytics-8.3.92:
  Successfully uninstalled ultralytics-8.3.92


In [ ]:
from ultralytics import YOLO

print("✅ YOLOv8 Imported Successfully!")

✅ YOLOv8 Imported Successfully!


In [ ]:
from ultralytics import YOLO

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/datasets/data.yaml epochs=5 imgsz=640 batch=16 device=0

100% 21.5M/21.5M [00:00<00:00, 168MB/s]
Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/datasets/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

In [ ]:
!cp runs/detect/train/weights/best.pt /content/best_model.pt
print("Trained YOLOv8 model saved as `best_model.pt`")

Trained YOLOv8 model saved as `best_model.pt`


In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("/content/best_model.pt")

# Print model summary
model.info()

Model summary: 129 layers, 11,137,535 parameters, 0 gradients, 28.7 GFLOPs


(129, 11137535, 0, 28.6555648)

In [ ]:
!yolo task=detect mode=val model=/content/best_model.pt data=/content/datasets/data.yaml

Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/labels/val.cache... 432 images, 0 backgrounds, 0 corrupt: 100% 432/432 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 27/27 [00:10<00:00,  2.47it/s]
                   all        432       4354      0.754      0.594      0.689       0.45
                   car        313       2702      0.811      0.794      0.867      0.594
                   bus        125        292       0.55      0.586      0.579      0.381
                person        264       1136      0.884      0.673      0.808      0.529
               bicycle         45         81        0.7      0.519      0.581      0.384
             motorbike         66        143      0.826      0.399      0.612      0.363
Speed: 1.4ms preprocess, 7.6ms inference, 0.0ms loss, 3.

In [ ]:
from ultralytics import YOLO

# Try loading the trained model
try:
    model = YOLO("/content/best_model.pt")
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)

Model loaded successfully!


In [ ]:
model.info()

Model summary: 129 layers, 11,137,535 parameters, 0 gradients, 28.7 GFLOPs


(129, 11137535, 0, 28.6555648)

In [ ]:
!ls /content/datasets/images/test/ | head -10

AM_Bing_211.png
AM_Google_250.png
BD_Baidu_105.png
BD_Baidu_117.png
BD_Baidu_199.png
BD_Baidu_211.png
BD_Baidu_284.png
BD_Baidu_461.png
BD_Baidu_476.png
BD_Baidu_486.png


In [ ]:
!rm -rf /content/runs/detect/*  # Deletes old predictions
!yolo task=detect mode=predict model=/content/best_model.pt \
      source=/content/datasets/images/test save=True \
      project=/content/runs name=detect

Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs

image 1/433 /content/datasets/images/test/AM_Bing_211.png: 384x640 2 persons, 42.1ms
image 2/433 /content/datasets/images/test/AM_Google_250.png: 448x640 4 persons, 41.3ms
image 3/433 /content/datasets/images/test/BD_Baidu_105.png: 480x640 4 cars, 2 buss, 40.5ms
image 4/433 /content/datasets/images/test/BD_Baidu_117.png: 288x640 4 cars, 1 bus, 1 person, 1 bicycle, 40.2ms
image 5/433 /content/datasets/images/test/BD_Baidu_199.png: 512x640 2 persons, 45.9ms
image 6/433 /content/datasets/images/test/BD_Baidu_211.png: 480x640 2 cars, 13.8ms
image 7/433 /content/datasets/images/test/BD_Baidu_284.png: 640x640 3 persons, 1 bicycle, 2 motorbikes, 16.9ms
image 8/433 /content/datasets/images/test/BD_Baidu_461.png: 512x640 16 cars, 13.8ms
image 9/433 /content/datasets/images/test/BD_Baidu_476.png: 384x640 8 persons, 11.3ms
image 10/43

In [ ]:
!ls /content/ultralytics/ultralytics/runs/detect/

ls: cannot access '/content/ultralytics/ultralytics/runs/detect/': No such file or directory


In [ ]:
!ls /content/ultralytics/ultralytics/runs/detect/predict3/

ls: cannot access '/content/ultralytics/ultralytics/runs/detect/predict3/': No such file or directory


In [ ]:
import glob
import IPython.display as display
from PIL import Image

# Load the predicted images
predicted_images = glob.glob("/content/ultralytics/ultralytics/runs/detect/predict3/*.jpg")

if len(predicted_images) == 0:
    print("No prediction images found! Check the file path.")
else:
    print(f"Found {len(predicted_images)} predicted images. Displaying now...")

    # Display the first 5 images
    for img_path in predicted_images[:5]:
        display.display(Image.open(img_path))

No prediction images found! Check the file path.


Deformable Convolution (C2f-DCN)

In [ ]:
import torch
import torch.nn as nn
from torchvision.ops import DeformConv2d

class C2fDCN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(C2fDCN, self).__init__()
        self.conv_offset = nn.Conv2d(in_channels, 18, kernel_size=kernel_size, padding=1)
        self.dcn = DeformConv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)

    def forward(self, x):
        offsets = self.conv_offset(x)
        x = self.dcn(x, offsets)
        return x

Involution

In [ ]:
class Involution(nn.Module):
    def __init__(self, in_channels, kernel_size=3):
        super(Involution, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, groups=in_channels, padding=1)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x

FasterNet

In [ ]:
class FasterNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FasterNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return x

S5 Attention

In [ ]:
class S5Attention(nn.Module):
    def __init__(self, channels):
        super(S5Attention, self).__init__()
        self.attn = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=1),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return x * self.attn(x)

In [ ]:
!pip install ultralytics --upgrade --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

 Integrate C2f-DCN, FasterNet, Involution, and S5Attention into YOLOv8

In [ ]:
from ultralytics.models.yolo import Model

class YOLOv8DF(Model):
    def __init__(self, cfg="yolov8s.yaml"):
        super().__init__(cfg)

        # Modify backbone
        self.model[0] = C2fDCN(3, 64)  # Replace first layer with Deformable Convolution

        # Modify feature extraction
        self.model[3] = FasterNet(64, 128)
        self.model[5] = Involution(128)

        # Modify attention mechanism
        self.model[-2] = S5Attention(256)

        print("✅ YOLOv8-DF Model Initialized with Improvements")

# Load and print model summary
model = YOLOv8DF()
print(model)

Train the Improved YOLOv8-DF Model

In [ ]:
!yolo task=detect mode=train model=yolov8_df.pt data=/content/datasets/data.yaml epochs=20 imgsz=640 batch=16 device=0


Validate Model Performance

In [ ]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/datasets/data.yaml

Test YOLOv8-DF on Foggy Images

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt source=/content/datasets/images/test save=True

 Display Predictions from the Improved Model

In [ ]:
import glob
import IPython.display as display
from PIL import Image

# Load images from detect/
predicted_images = glob.glob("/content/runs/detect/*.jpg")

if len(predicted_images) == 0:
    print("❌ No prediction images found! Check YOLOv8 inference output.")
else:
    print(f"✅ Found {len(predicted_images)} predicted images. Displaying now...")

    # Display first 5 images
    for img_path in predicted_images[:5]:
        display.display(Image.open(img_path))